# Copy instead of modifying to preserve attack parameters

In [1]:
AGENT_NAME = '20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SURROGATE_PATH = '20 bin PPO 500 results\surrogates\imitator.pth'
#RUNS = 5
TRIAL = 'toggle bifurcated DL loss'
SAVE_DIR = '20 bin PPO 500 results\multi eps vs asr for imitator FGM' + '/'

In [2]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.estimators.classification import PyTorchClassifier as classifier
from art.attacks.evasion import FastGradientMethod as FGM

import pandas as pd
import numpy as np
#import os
import torch
#from torch.nn import CrossEntropyLoss

import KBMproject.utilities as utils

from joblib import Parallel, delayed

%matplotlib inline

In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
testilons = np.arange(0.02, 0.21, 0.01)
RUNS = len(testilons)

Define RL agent

In [5]:
agents = []
for _ in range (RUNS):
    agents.append(PPO.load(AGENT_NAME))

Create environments

In [6]:
envs = []
for _ in range (RUNS):
    envs.append(utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42))

In [7]:
cols = utils.make_discrete_env(schema=schema,  
                            action_bins=agents[0].action_space[0].n,
                            seed=42).observation_names

In [8]:
imitator = torch.load(SURROGATE_PATH)

In [9]:
observation_masks = np.ones(agents[0].observation_space.shape)
observation_masks[0:6] = 0 #mask time features
print('masked features:')
cols[0][0:6]

masked features:


['month_cos',
 'month_sin',
 'day_type_cos',
 'day_type_sin',
 'hour_cos',
 'hour_sin']

In [10]:
kwargs = dict(norm=np.inf,
              targeted=True, #for toggle targeted attack
        #eps=0.05,
        #num_random_init=2,
        )
attacks =[]
for run in range(RUNS):
    attacks.append(FGM(estimator=classifier(
                                        model=utils.MaximumBifuricationWrapper(imitator), #add MaximumBifuricationWrapper?
                                        loss=utils.CWLoss(),
                                        #nb_classes=agents[run].action_space[0].n, 
                                        nb_classes=2,
                                        input_shape=agents[run].observation_space.shape,
                                        device_type='gpu',
                                        clip_values = (agents[run].observation_space.low.min(),
                                                       agents[run].observation_space.high.max()),
                                        ),
                        eps=testilons[run],
                        **kwargs))

In [11]:
%%time
%%capture
results = Parallel(n_jobs=RUNS, verbose=10, prefer='threads')(delayed(
    utils.eval_toggle_targeted_attack)(agent, env, attack) for agent, env, attack in zip(agents, envs, attacks)) 


CPU times: total: 5min 35s
Wall time: 17min 25s


Results is a list of tupples for each run, of the format(KPIs, observations, perturbed observations, epsilons)

In [12]:
kpis = [results[i][0] for i in range(len(results))]
df_kpis = pd.concat(kpis, axis='columns',keys=testilons)

In [13]:
#df_kpis[['mean', 'std', 'variance']] = df_kpis.agg(['mean','std', 'var'], axis='columns')

In [14]:
df_kpis

,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20
cost_function,,,,,,,,,,,,,,,,,,,
annual_peak_average,1.009338,1.009338,1.009338,1.037420,1.037393,1.037361,1.241576,1.342564,1.176343,1.176310,1.176254,1.158283,1.034155,1.158283,1.287562,1.287562,1.287562,1.287562,1.225536
carbon_emissions_total,0.876263,0.880781,0.884486,0.891711,0.900988,0.910214,0.920135,0.931527,0.938889,0.946875,0.961298,0.973406,0.981754,0.999288,1.011297,1.028492,1.042561,1.057012,1.068697
cost_total,0.794713,0.801355,0.806567,0.820862,0.832464,0.846085,0.860040,0.875884,0.886932,0.899968,0.917252,0.933194,0.944130,0.965590,0.982131,0.999073,1.013719,1.032331,1.047008
daily_one_minus_load_factor_average,1.075350,1.063876,1.071858,1.068670,1.032892,0.997187,0.987693,0.989346,0.983365,0.979007,0.976165,0.968801,0.971541,0.970573,0.973616,0.972197,0.971335,0.971097,0.971573
daily_peak_average,0.903369,0.915465,0.921102,0.950269,0.977014,1.013005,1.041031,1.085523,1.103913,1.133522,1.158801,1.186268,1.202036,1.241405,1.293272,1.317741,1.352546,1.366861,1.367306
electricity_consumption_total,0.886466,0.890253,0.893837,0.899920,0.907971,0.915762,0.924381,0.934064,0.940604,0.947598,0.959943,0.972190,0.980665,0.997824,1.009559,1.026790,1.039901,1.053526,1.064835
monthly_one_minus_load_factor_average,0.975979,0.975868,0.975680,0.987737,0.989718,0.992146,0.996427,0.999321,0.991126,0.994470,0.993610,0.998416,0.996453,0.998924,1.006946,1.006160,1.007985,1.007616,1.011074
ramping_average,1.141118,1.167647,1.202078,1.266443,1.320452,1.394736,1.466110,1.538377,1.601698,1.675659,1.770372,1.859683,1.920178,2.034144,2.131163,2.239348,2.315974,2.412207,2.472087
zero_net_energy,1.108449,1.109574,1.110783,1.113532,1.115522,1.117939,1.120688,1.124189,1.126234,1.128731,1.131841,1.134339,1.136648,1.139774,1.142837,1.145514,1.147729,1.151485,1.152944


In [15]:
df_kpis.to_csv(SAVE_DIR + f'{RUNS} run KPIs {TRIAL}.csv')

FGSM using the same parameters in a whitebox attack has an ASR of 0.7.

Increasing the number of restarts decreases the ASR, likely because of differences between the surrogate and vicitm models

In [16]:
ASRs = [results[i][3] for i in range(len(results))]
#print(f'For 5 runs of the random attack \nthe mean ASR is: {np.mean(ASRs):.3f}\nthe STD is: {np.std(ASRs):.3f}')

In [17]:
ASRs

[0.17479164288160748,
 0.24123758419910948,
 0.292727480305971,
 0.3395364767667542,
 0.37812535677588766,
 0.41020664459413175,
 0.4401187350154127,
 0.46603493549491953,
 0.48669939490809455,
 0.5067930129010161,
 0.5309966891197625,
 0.55040529740838,
 0.5675305400159836,
 0.5797465464094075,
 0.5993834912661262,
 0.617193743578034,
 0.6305514328119648,
 0.6429957757734901,
 0.6549834455988126]

In [18]:
pd.DataFrame({'ASRs':ASRs}, index=testilons,).to_csv(SAVE_DIR + f'ASR for varied epsilons {TRIAL}.csv')

In [19]:
for run in range(RUNS):
        pd.DataFrame(results[run][1],columns=cols,).to_csv(SAVE_DIR+f'run {run} obs {TRIAL}.csv',)
        pd.DataFrame(results[run][2],columns=cols,).to_csv(SAVE_DIR+f'run {run} adv obs {TRIAL}.csv')